In [2]:
from gurobipy import *
import math
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import savefig 

In [3]:
#Define Model 
m = Model()

Using license file C:\Users\Anojan Logeswaran\gurobi.lic
Academic license - for non-commercial use only


In [4]:
#Read csv file and clean data
df = pd.read_csv('Capstone_Demand_Data_Aerial_View.csv')
df.fillna(0)
df.columns = df.columns.str.lower().str.replace(' ','_')

In [5]:
#Create x,y coordinates for grid points
coordinates = []
for i in range (5,200,10):
    for j in range (5,200,10):
        newlist = [i,j]
        coordinates.append(newlist)

In [6]:
#Creating and appending coordinates to dataframe 
a=[]
b=[]
c=[]

for item in coordinates: 
    for i in item:
        c.append(i)
        
for i in range(0,799,2):
    a.append(c[i])
    b.append(c[i+1])

df['x_coordinate']=a
df['y_coordinate']=b

In [7]:
#Creating sample dataframes from main dataframe to divide information appropiately
family_information = df[['cell','demand','x_coordinate','y_coordinate']].copy()
cell_demand = family_information
potential_wells = df[['cell','potential_water_access_point','x_coordinate','y_coordinate']].copy()

#Number of water access points the user wants
number_of_wells = df.iloc[0]['how_many_water_access_points_do_you_want?_(minimum_1)']

#Current location of well
current_well_location = df.iloc[0]['enter_the_cell_locations_of_any_existing_wells.']

In [8]:
#Extracts x,y coordinates based on current location of well 
current_well_x_coordinate = family_information.loc[family_information['cell'] == current_well_location,'x_coordinate'].iloc[0]
current_well_y_coordinate = family_information.loc[family_information['cell'] == current_well_location,'y_coordinate'].iloc[0]

In [9]:
#Using family points where demand is > 0 
family_information = family_information[family_information.demand>0]

#Only using wells that are confirmed as Yes
potential_wells = potential_wells[potential_wells.potential_water_access_point == 'Y']

In [10]:
potential_wells.reset_index(drop=True)
family_information.reset_index(drop=True)

,cell,demand,x_coordinate,y_coordinate
0,B1,100,15,5
1,B2,100,15,15
2,B19,100,15,185
3,C1,400,25,5
4,C2,200,25,15
5,C3,200,25,25
6,C4,200,25,35
7,C5,100,25,45
8,C6,400,25,55
9,C7,300,25,65


In [11]:
#Coverting coordinates of families with demand > 0 to lists 
a = family_information.x_coordinate.tolist()
b = family_information.y_coordinate.tolist()

#Creating and appending a list to hold the coordinates as a list of lists 
family_coordinates = []
for i in range(0,len(a)):
    test = a[i]
    test2 = b[i]
    new_list = [test,test2]
    family_coordinates.append(new_list)

In [12]:
#Coverting coordinates of potential wells to lists 
c = potential_wells.x_coordinate.tolist()
d = potential_wells.y_coordinate.tolist()

#Creating and appending a list to hold the coordinates as a list of lists 
well_coordinates = []
for i in range(0,len(c)):
    test = c[i]
    test2 = d[i]
    new_list = [test,test2]
    well_coordinates.append(new_list)

In [13]:
#Identifying where in the list is the current well coordinates located - to be used later when setting binary variable value to 1
current_well_index = well_coordinates.index([current_well_x_coordinate,current_well_y_coordinate])

In [14]:
families = family_coordinates
wells = well_coordinates

In [15]:
# Defining function to calculate euclidean distance 
def distance(a,b):
  dx = a[0] - b[0]
  dy = a[1] - b[1]
  return math.sqrt(dx*dx + dy*dy)

In [16]:
# Add variables
x = {}
y = {}
d = {} # Distance matrix (not a variable)
p = number_of_wells #number of water access points 

In [17]:
#Create X as a binary variable (dependent on number of wells to choose from)
for j in range(len(wells)):
    x[j] = m.addVar(vtype=GRB.BINARY)

In [18]:
#By setting upper bound and lower bound as 1, this ensures that the binary variable corresponding to the current well is set as 1
x[current_well_index].setAttr(GRB.Attr.UB, 1.0)
x[current_well_index].setAttr(GRB.Attr.LB,1.0)
m.update()

In [19]:
#calculating distance between wells and families
for i in range(len(families)):
    for j in range(len(wells)):
        y[(i,j)] = m.addVar(vtype=GRB.BINARY)
        d[(i,j)] = distance(families[i], wells[j])

In [20]:
m.update()

In [21]:
#Add Constraint - Each family is assigned to at most 1 access point.
for i in range(len(families)):
  m.addConstr(quicksum(y[(i,j)] for j in range(len(wells))) == 1)

In [22]:
#Add Constraint - Exactly P access points are located
m.addConstr(quicksum(x[j] for j in range(len(wells))) == p)

<gurobi.Constr *Awaiting Model Update*>

In [23]:
# Add constraint - Ensures residents are not allocated to access points that do not exist.
for i in range(len(families)):
  for j in range(len(wells)):
    m.addConstr(y[(i,j)] <= x[j])

In [24]:
m.update()

In [25]:
# Set Objective 
m.setObjective( quicksum(d[(i,j)]*y[(i,j)] 
                         for i in range(len(families)) for j in range(len(wells))),GRB.MINIMIZE)

In [26]:
m.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 64077 rows, 64214 columns and 191980 nonzeros
Model fingerprint: 0x471538af
Variable types: 0 continuous, 64214 integer (64214 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 13869.442120
Presolve removed 47417 rows and 47230 columns
Presolve time: 1.02s
Presolved: 16660 rows, 16984 columns, 49926 nonzeros
Variable types: 0 continuous, 16984 integer (16984 binary)

Root relaxation: objective 9.329994e+03, 18915 iterations, 1.43 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    9329.9940175 9329.99402  0.00%     -    2s

Explored 0 nodes (18915 simplex iterations) in 2.61 seconds
Thread count was 4 (of 4 available proc

In [27]:
#appending values from model to an array, 1 means the well will be used, 0 means it will not
sample = []
for a,b in enumerate(m.getVars()):
    if (a == len(wells)):
        break
    else:
        sample.append(b.x)
        

In [33]:
len(m.getVars())

64214

In [ ]:
#only working with wells that were picked by the model 
potential_wells['model_results'] = sample
well_results = potential_wells[potential_wells['model_results'] > 0]

In [ ]:
well_results

In [ ]:
#Converting x,y coordinates of model results to list
c = well_results.x_coordinate.tolist()
d = well_results.y_coordinate.tolist()

#Appending coordinates to a list 
final_well_coordinates = []
for i in range(0,len(c)):
    test = c[i]
    test2 = d[i]
    new_list = [test,test2]
    final_well_coordinates.append(new_list)

In [ ]:
#Calculating distance between families and wells given by model
d = {}
for i in range(len(families)):
    for j in range(len(final_well_coordinates)):
        d[(i,j)] = distance(families[i], final_well_coordinates[j])

In [ ]:
#of the wells given, chooses and assigns wells to families based on shortest distance 
Optimal_well_distance = []
dummy_variable = 0
Optimal_well_number = []
for i in range(0,len(families)):
    min_distance = 10000000
    for j in range(0,len(final_well_coordinates)):
        if d[i,j] < min_distance:
            min_distance = d[i,j]
            dummy_variable = j
        if j ==(len(final_well_coordinates) -1):
            Optimal_well_distance.append(min_distance)
            Optimal_well_number.append(dummy_variable)

In [ ]:
final_results = family_information[['cell','x_coordinate','y_coordinate']].copy()

In [ ]:
well_results.reset_index(drop=True, inplace=True)

In [ ]:
Optimal_well_coordinate = []
for i in Optimal_well_number:
    dummy = well_results.cell.loc[i]
    Optimal_well_coordinate.append(dummy)

In [ ]:
final_results['assigned_well'] = Optimal_well_coordinate
final_results['distance_to_assigned_well'] = Optimal_well_distance

In [ ]:
final_results['distance_to_assigned_well'] = 20 * final_results['distance_to_assigned_well']
final_results
final_results.to_csv('distances.csv',index=False)

In [ ]:
final_distances = final_results['distance_to_assigned_well']
final_demands = family_information['demand']

final_demand_weighted_distance = final_distances * final_demands
final_results['demand_weighted_distance'] = final_demand_weighted_distance

In [ ]:
final_results

In [ ]:
final_results[['letters', 'numbers']] = final_results['cell'].str.extract('([A-Za-z]+)(\d+\.?\d*)', expand = True)
final_results_v2 = final_results[['cell','letters','numbers','distance_to_assigned_well']].copy()
final_results_v2 
all_cells = df[['cell','demand']].copy()
all_cells[['letters','numbers']] = all_cells['cell'].str.extract('([A-Za-z]+)(\d+\.?\d*)', expand = True)

In [ ]:
# Final Dataframe
zero_demand_cells = all_cells[all_cells.demand == 0]
zero_demand_cells = zero_demand_cells[['cell','letters','numbers','demand']]
zero_demand_cells = zero_demand_cells.rename(columns={'demand':'distance_to_assigned_well'})
zero_demand_cells
final_results_v3 = pd.concat([zero_demand_cells,final_results_v2])
final_results_v3 = final_results_v3.sort_index(ascending=True)

In [ ]:
# Create empty list of lists
listOfLists = []
# Create counter
x = 0
# Append lists to list of lists
for i in range(20):
    tempList = []
    for j in range (20):
        tempList.append(final_results_v3['distance_to_assigned_well'].values[x])
        x = x + 1
    listOfLists.append(tempList)

In [ ]:
# Convert list of lists to array and transpose
array = np.array(listOfLists)
array
transposeArray = array.T

In [ ]:
# Create heatmap
shape = transposeArray.shape
labels = np.zeros(shape)
x_axis_labels = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T']
y_axis_labels = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
heatmap_distance = sns.heatmap(transposeArray, vmax=2500, xticklabels=x_axis_labels, yticklabels=y_axis_labels, linewidths=1, linecolor='black', cmap="Reds")
heatmap_distance_figure = heatmap_distance.get_figure()
heatmap_distance_figure.savefig('heatmap_distance.png', dpi=400)
#final_results_v2
#transposeArray
well_results


In [ ]:
# Create empty list of lists for demand
listOfLists_demand = []
# Create counter
x = 0
# Append lists to list of lists
for i in range(20):
    tempList = []
    for j in range (20):
        tempList.append(cell_demand['demand'].values[x])
        x = x + 1
    listOfLists_demand.append(tempList)

In [ ]:
# Convert demand list of lists to array and transpose
array_demand = np.array(listOfLists_demand)
array_demand
transposeArray_demand = array_demand.T

In [ ]:
# Create heatmap for demand
shape_v2 = transposeArray_demand.shape
labels_v2 = np.zeros(shape)
heatmap_demand = sns.heatmap(transposeArray_demand, xticklabels=x_axis_labels, yticklabels=y_axis_labels, linewidths=1, linecolor='black', cmap="Reds")
heatmap_demand_figure = heatmap_demand.get_figure()
heatmap_demand_figure.savefig('heatmap_demand.png', dpi=400)